In [9]:
import requests
import json

# URL to fetch data from
url = "https://freetestapi.com/api/v1/books"

# Send a GET request to the URL
response = requests.get(url)

# Convert the response to JSON
data = response.json()

# Open or create data.json then write the data
with open('book_test.json', 'w') as f:
    json.dump(data, f)

Data for books

In [2]:
import os
import glob

# Open the 'book_imgs' folder
os.chdir('/home/admin88/Desktop/workspace/ISD.ICT.20232.22/data/book_imgs')

# Filter all files
all_files = glob.glob('*')

# Filter all images
images = glob.glob('*.webp') + glob.glob('*.jpg') + glob.glob('*.jpeg') + glob.glob('*.png')

# Create a set of image file names
image_files = set(images)

# Iterate over all files
for file in all_files:
    # If the file is not an image, delete it
    if file not in image_files:
        os.remove(file)
        
# Create a set to store the first 11 characters of image names
image_names = set()

# Iterate over all images
for image in images:
    # If the first 11 characters of the image name are already in the set, delete the image
    if image[:11] in image_names:
        os.remove(image)
    else:
        # Otherwise, add the first 11 characters of the image name to the set
        image_names.add(image[:11])

Data for CDs

In [4]:
import os
import glob

# Open the 'cd_imgs' folder
os.chdir('/home/admin88/Desktop/workspace/ISD.ICT.20232.22/data/CD_imgs')

# Filter all files
all_files = glob.glob('*')

# Filter all images
images = glob.glob('*.webp') + glob.glob('*.jpg') + glob.glob('*.jpeg') + glob.glob('*.png')

# Create a set of image file names
image_files = set(images)

# Iterate over all files
for file in all_files:
    # If the file is not an image, delete it
    if file not in image_files:
        os.remove(file)
        
# Create a set to store the first 11 characters of image names
image_names = set()

# Iterate over all images
for image in images:
    # If the first 8 characters of the image name are already in the set, delete the image
    if image[:8] in image_names:
        os.remove(image)
    else:
        # Otherwise, add the first 8 characters of the image name to the set
        image_names.add(image[:8])

In [5]:
import os
import glob 

# Open the 'dvd_imgs' folder
os.chdir('/home/admin88/Desktop/workspace/ISD.ICT.20232.22/data/dvd_imgs')

# Filter all files
all_files = glob.glob('*')

# Filter all images
images = glob.glob('*.webp') + glob.glob('*.jpg') + glob.glob('*.jpeg') + glob.glob('*.png')

# Create a set of image file names
image_files = set(images)

# Iterate over all files
for file in all_files:
    # If the file is not an image, delete it
    if file not in image_files:
        os.remove(file)
        
# Create a set to store the first 11 characters of image names
image_names = set()

# Iterate over all images
for image in images:
    # If the first 8 characters of the image name are already in the set, delete the image
    if image[:8] in image_names:
        os.remove(image)
    else:
        # Otherwise, add the first 8 characters of the image name to the set
        image_names.add(image[:8])